# Capstone Project

## Introduction/Business Problem

I will be investigating potential Postal Code (neighborhoods) to open a new startup company in Toronto, Ontario, Canada. My goal is to identify Postal Code (neighborhoods) outside downtown Toronto with similar access to necessary amenities. This is to avoid the high cost of renting a location for a startup company.





## Data


1- Scrape https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to find the list of neighborhoods in Toronto and their postal code.



2- scrape https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto to find the connection between neighborhoods and district numbers.



3- Get the pricing information for each district number form http://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_0420.pdf (April 2020 update).



4- Store data in Pandas dataframe and do data cleanup and assign a district number and price to each Postal Code.



5- Geolocate neighborhood postal codes and find "lat" and "long" values for each.



6- Use Foursquare to explore different postal codes in Toronto with radius of 500 and limit of 100. 



7- Analyze each Postal Code and create a dataframe and display the top 20 venues for each Postal Code.



8- Cluster Postal Codes and run k-means to cluster the Postal Codes into 20 clusters.



9- Create a new dataframe that includes original scrapped data and the clusters as well as the top 10 venues for each Postal Code.



10- Visualize the dataframe using Folium mapping package.



11- Conclusion.



In [1]:
# Import required libraries

#Install the following if they have not been installed already by uncommenting them
!conda install -c conda-forge beautifulsoup4 bs4 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge folium=0.5.0 --yes

import numpy as np 
import pandas as pd 
import urllib.request, urllib.parse, urllib.error # import urllib library parts needed
import bs4 as bs # import beautifulsoup library for web scrapping
import ssl # Import SSL certificatation library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # import library to tranform JSON file into a pandas dataframe

import folium # import map rendering library
from sklearn.cluster import KMeans # import k-means from clustering stage


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

## Scrape the following Wikipedia page to get a list of postal codes and their neighborhoods and boroughs in Toronto:
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [ ]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


html = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', context=ctx).read()
soup = bs.BeautifulSoup(html, 'html.parser')

# Retrieve all of the table rows and put them into a pandas dataframe
row_list = []
table_rows = soup('tr')
for row in table_rows:
    td = row.find_all('td')
    row = [tr.text for tr in td]
    row_list.append(row)
Toronto_DF = pd.DataFrame(row_list)

# Process the scraped dataframe to satisfy the following requirements:
#1- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.  
#2- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.  
#3- More than one neighborhood can exist in one postal code area.  
#4- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.  


#Clean up the dataframe

#Just keep required columns
Toronto_PostalCode_Neighborhood = Toronto_DF[[0,1,2]]

#Rename columns
Toronto_PostalCode_Neighborhood.columns=["PostalCode", "Borough", "Neighborhood"]

#Remove any '\n' in the strings
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood.replace('\n','', regex=True)

#Drop rows that are not required
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood.dropna()
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Borough != 'Not assigned']
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Borough != 'B' ]
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Borough != 'NL']
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Borough != 'NS']
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Borough != 'Mississauga']
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood[Toronto_PostalCode_Neighborhood.Neighborhood != 'Stn A PO Boxes']


#Group rows if required
Toronto_PostalCode_Neighborhood['Neighborhood'] = Toronto_PostalCode_Neighborhood.groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
Toronto_PostalCode_Neighborhood = Toronto_PostalCode_Neighborhood.drop_duplicates()

Toronto_PostalCode_Neighborhood


## Transform the dataframe to have neighborhoods in rows
This is required as in the following steps we will try to assign an average price to each postal code based on its neighborhoods

In [ ]:
#Make a copy of "Toronto_PostalCode_Neighborhood" dataframe and call it "Toronto_DF"
Toronto_DF = Toronto_PostalCode_Neighborhood 

#create temoporary column and combine Postalcode and Borough in it
Toronto_DF["TEMP_CONCAT"] = Toronto_DF["PostalCode"] + "--" + Toronto_DF["Borough"]

#split the neighborhoods and put them in rows along with their coresponding temp column value
Toronto_DF = pd.concat([pd.Series(row['TEMP_CONCAT'], row['Neighborhood'].split(','))              
                    for _, row in Toronto_DF.iterrows()]).reset_index()
#rename the collumns
Toronto_DF.columns = ["Neighbourhood","TEMP_CONCAT"] 

#split the temp column to the original Postalcode and Borough columns
Toronto_DF[['PostalCode','Borough']] = Toronto_DF.TEMP_CONCAT.str.split('--', expand=True)

#drop the temp column
Toronto_DF.drop(columns=["TEMP_CONCAT"], inplace=True)

#remove any leading or trailing spaces from string columns
df_obj = Toronto_DF.select_dtypes(['object'])
Toronto_DF[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

Toronto_DF

## Scrape the following Wikipedia page to get a list MLS districts and their neighborhoods:
https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Multiple_listing_service_districts_and_neighbourhoods

To understand MLS district you can use this map:
https://www.torontomls.net/Communities/map.html

In [ ]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


html = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto#Multiple_listing_service_districts_and_neighbourhoods', context=ctx).read()
soup = bs.BeautifulSoup(html, 'html.parser')

# Retrieve table headers and all of the table rows and put them into a pandas dataframe

#Find the table
Wiki_table = soup.find("table", attrs={"class": "wikitable"})
table_rows = Wiki_table.tbody.find_all("tr")

#Headers
header = []
th = table_rows[0].find_all('th')
header.append(header)
header = [tr.text.rstrip() for tr in th]

#rows
row_list = []
for row in table_rows:
    td = row.find_all('td')
    row = [tr.text.rstrip() for tr in td]
    row_list.append(row)
Toronto_District_Neighbourhood_DF = pd.DataFrame(row_list, columns= header)

#Drop rows that are not required
Toronto_District_Neighbourhood_DF = Toronto_District_Neighbourhood_DF.dropna()




Toronto_District_Neighbourhood_DF = pd.concat([pd.Series(row['District Number'], row['Neighbourhoods Included'].split(','))              
                    for _, row in Toronto_District_Neighbourhood_DF.iterrows()]).reset_index()

Toronto_District_Neighbourhood_DF.columns = ["Neighbourhood","District Number"] 

Toronto_District_Neighbourhood_DF['Neighbourhood'] = Toronto_District_Neighbourhood_DF['Neighbourhood'].map(lambda x: x.strip())


Toronto_District_Neighbourhood_DF

## Get the housing composite benchmark prices for each district from the following page as of April 2020:
http://trreb.ca/files/market-stats/home-price-index/TREB_MLS_HPI_Public_Tables_0420.pdf

### Composite Price Benchmark:  
Benchmarks and indices within the “Composite” category represents all homes used in models,
including One- and Two-storey single family homes, Townhouses and Apartments. Accordingly, the
Composite index includes both attached and detached homes within One- and Two-storey single family
homes.

In [ ]:
#It was not possible to scrape the PDF directly, a text file manually created from the PDF and used to import the values
Toronto_District_Price_DF = pd.read_csv('Price.txt', sep=",,,", header=0)

Toronto_District_Price_DF = Toronto_District_Price_DF.iloc[:,0:3:2]

#Remove any ',' in the strings
Toronto_District_Price_DF = Toronto_District_Price_DF.replace(',','', regex=True)

#Convert the type of column "Composite Price Benchmark" to numeric 
Toronto_District_Price_DF['Composite Price Benchmark'] = Toronto_District_Price_DF['Composite Price Benchmark'].astype('int64') 


Toronto_District_Price_DF

## Link MLS districts and their neighborhoods to the prices
From here forward, we will assume that each neighborhood in a district has the same price as that district.

In [ ]:
#Join "MLS districts and their neighborhoods" to "districts housing composite benchmark prices" based on District Number
Toronto_District_Neighbourhood_Price_DF = Toronto_District_Neighbourhood_DF.join(Toronto_District_Price_DF.set_index('District Number'), on='District Number')

Toronto_District_Neighbourhood_Price_DF

## Having a price for each neighborhood, we can find an average price for each Postal code as we have the neighborhood information per Postal code as well

In [ ]:
#Join "MLS districts and their neighborhoods with assigned prices" to "Toronto_DF" based on Neighbourhood
Toronto_Neighborhood_District_Price_PostalCode_DF = Toronto_DF.join(Toronto_District_Neighbourhood_Price_DF.set_index('Neighbourhood'), on='Neighbourhood')

#Drop rows with no price
Toronto_Neighborhood_District_Price_PostalCode_DF = Toronto_Neighborhood_District_Price_PostalCode_DF.dropna()

#Sort rows based on PostalCode column
Toronto_Neighborhood_District_Price_PostalCode_DF.sort_values(['PostalCode'])

# Find Average Price for each postal code
Toronto_AveragePrice_Per_PostalCode = Toronto_Neighborhood_District_Price_PostalCode_DF.groupby('PostalCode')['Composite Price Benchmark'].mean().reset_index()
Toronto_AveragePrice_Per_PostalCode 

## Join these prices to the initial "Toronto_PostalCode_Neighborhood" dataframe

In [ ]:
#Join "Toronto_PostalCode_Neighborhood" to "Toronto_AveragePrice_Per_PostalCode" based on Postalcode
Toronto_PostalCode_Neighborhood_Price = Toronto_PostalCode_Neighborhood.join(Toronto_AveragePrice_Per_PostalCode.set_index('PostalCode'), on='PostalCode')

Toronto_PostalCode_Neighborhood_Price

## Get the latitude and the longitude coordinates of each postal code, using geocoder:

In [ ]:
# import geocoder
import geocoder 

# Define a function to convert PostalCode to lat and Long
def Postal_Code_to_LatLong(postal_code):
    
 # initialize your variable to None
 lat_lng_coords = None
    
    
# Loop until you get the coordinates
 while(lat_lng_coords is None):
     g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
     lat_lng_coords = g.latlng
 return lat_lng_coords

# Use the defined function to add latitude and longitude for each postal code
Toronto_PostalCode_Neighborhood_Price[['Latitude', 'Longitude']] =  pd.DataFrame(Toronto_PostalCode_Neighborhood_Price['PostalCode'].apply(Postal_Code_to_LatLong).to_list(), index=Toronto_PostalCode_Neighborhood_Price.index)

Toronto_PostalCode_Neighborhood_Price

## Define Foursquare Credentials and Version:

In [ ]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [ ]:
# The code was removed by Watson Studio for sharing.

## Create a function to get nearby venues for each PostalCode:

In [ ]:
def getNearbyVenues(PostalCode, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for PostalCode, lat, lng in zip(PostalCode, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            PostalCode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'PostalCode Latitude', 
                  'PostalCode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Run the above function on each PostalCode in Toronto and create a new dataframe called "toronto_venues":

In [ ]:
toronto_venues = getNearbyVenues(PostalCode=Toronto_PostalCode_Neighborhood_Price['PostalCode'],
                                   latitudes=Toronto_PostalCode_Neighborhood_Price['Latitude'],
                                   longitudes=Toronto_PostalCode_Neighborhood_Price['Longitude']
                                  )

#remove the row with venue category identified as 'Neighborhood' because it causes issues in the next steps
toronto_venues = toronto_venues[toronto_venues['Venue Category'] != 'Neighborhood' ]

#check the size of the resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()

## Analyze each record for Venue Categories by using one-hot encoding (dummy categories):

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")


# add neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


toronto_onehot

## Aggregate records at PostalCode level by taking the mean of the frequency of occurrence of each venue category:

In [ ]:
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped

## Function to sort the Venue Categories in descending order for each Postal Code based on mean frequencies calculated:

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Create a new dataframe and display the top 10 venue categories for each PostalCode (Using the above function):

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue Category'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue Category'.format(ind+1))

# create a new dataframe
PostalCode_Venue_Category_Sorted = pd.DataFrame(columns=columns)
# Add PostalCode Column to the new dataframe
PostalCode_Venue_Category_Sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    PostalCode_Venue_Category_Sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

PostalCode_Venue_Category_Sorted.head()

## Cluster PostalCodes, run k-means to cluster the Postalcodes into 20 clusters:

In [ ]:
# set number of clusters
kclusters = 20

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

## Create a new dataframe that includes the cluster as well as the top 10 venues for each PostalCode:

In [ ]:
# add clustering labels
PostalCode_Venue_Category_Sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_PostalCode_Neighborhood_Price

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(PostalCode_Venue_Category_Sorted.set_index('PostalCode'), on='PostalCode')

#Drop any Postalcode which was not ranked properly
toronto_merged.dropna(inplace=True)

toronto_merged

## Visualize the resulting clusters:

In [ ]:
# Get the latitude and longitude values of Toronto City
address = 'Toronto, ON, Canada'
location = geocoder.arcgis(address)
lat_lng_coords = location.latlng
latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, pc, poi, price, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'],toronto_merged['Neighborhood'],toronto_merged['Composite Price Benchmark'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(pc) + ' (' + str(poi) + ')' + ' Price ' + str(price) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= price/100000,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusion

Based on this simple analysis there are many areas in the City of Toronto with access to similar amenities as downtown area. The analysis is also illustrating that midtown area is more expensive than downtown area but uptown area has similar housing prices for locations with similar clustering. 